# Classifier les titres d’articles informant l’appartenance ou non au sujet climat

Afin de mesurer la couverture des sujets climats/environmentaux, voici différentes thématiques qu’on aimerait résoudre :
1. [**Binary classification**] Classifier les titres d’articles informant l’appartenance ou non au sujet climat.
2. [**Topic modelling**] Classifier le topic des articles afin de pouvoir comparer par example, la couverture du traitement de la reforme des retraites à la couverture de la sortie du rapport du GIEC.
3. Detecter les articles climato-sceptique et/ou anti-environnment (par example ventant les mérites de l’avion ou des températures douces en hiver).
4. [**Sentiment analysis**] Comprendre le ton des articles: par example: “Climat : nous avons encore les moyens d’agir” est positif et subjectif tandis que “Limiter le réchauffement à 1,5 °C ? Trop tard, affirment mille scientifiques” est négatif et objectif(?).

Dans ce notebook, on s'intéresse à la première tâche.

**Table of contents**

0. [Load data & EDA](#0-load-data--eda)
1. [Label sitemap data](#1-label-sitemap-data)


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Standard imports
import sys
sys.path.append('../..')
import json
import itertools

# Third-party imports
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from wordcloud import WordCloud
from nltk.stem import WordNetLemmatizer, SnowballStemmer
import gensim
import datetime

# Local imports
from quotaclimat.data_ingestion.config_sitmap import MEDIA_CONFIG, SITEMAP_CONFIG
from quotaclimat.data_processing.sitemap.sitemap_processing import load_all

## 0. Load data & EDA

In [21]:
# Load sitemap data
df = load_all("../../data_public/sitemap_dumps/")

In [4]:
print(f"Number of scraped news: {df.shape[0]}")
print(f"Number of features: {df.shape[1]}")

Number of scraped news: 243015
Number of features: 26


In [5]:
df.head()

,url,news,news_publication,publication_name,publication_language,news_publication_date,news_title,news_keywords,image,image_loc,...,media,section,changefreq,news_access,image_title,lastmod,news_genres,priority,download_date_last,media_type
32967,https://www.francetvinfo.fr/faits-divers/dispa...,\n,\n,Franceinfo,fr,2022-11-29 23:08:36,Disparitions en Isère : Yves Chatain a-t-il to...,"Faits-divers, France",\n,https://www.francetvinfo.fr/image/761gi5vt8-97...,...,francetvinfo,[faits-divers],None,None,None,None,None,NaN,2022-11-30 23:25:43,tv
32968,https://www.francetvinfo.fr/economie/emploi/ca...,\n,\n,Franceinfo,fr,2022-11-29 23:13:06,Réforme des retraites : vers un départ à la re...,"Retraite, Vie-professionnelle, Carrière, Emplo...",\n,https://www.francetvinfo.fr/image/761gi62g7-81...,...,francetvinfo,"[economie, emploi, carriere, vie-professionnel...",None,None,None,None,None,NaN,2022-11-30 23:25:43,tv
32969,https://www.francetvinfo.fr/politique/gouverne...,\n,\n,Franceinfo,fr,2022-11-29 23:25:38,Justice : Caroline Cayeux soupçonnée de fraude...,"Gouvernement d'Elisabeth Borne, Politique",\n,https://www.francetvinfo.fr/image/761gi693v-a0...,...,francetvinfo,"[politique, gouvernement-d-elisabeth-borne]",None,None,None,None,None,NaN,2022-11-30 23:25:43,tv
32970,https://www.francetvinfo.fr/france/hauts-de-fr...,\n,\n,Franceinfo,fr,2022-11-29 23:19:12,Effondrement d'immeubles à Lille : de nouveaux...,France,\n,https://www.francetvinfo.fr/image/761gi68c5-9c...,...,francetvinfo,"[france, hauts-de-france, nord, lille]",None,None,None,None,None,NaN,2022-11-30 23:25:43,tv
32971,https://www.francetvinfo.fr/economie/emploi/ca...,\n,\n,Franceinfo,fr,2022-11-29 23:22:22,Réforme des retraites : le gouvernement envisa...,"Retraite, Vie-professionnelle, Carrière, Emplo...",\n,https://www.francetvinfo.fr/image/761gi68q6-0d...,...,francetvinfo,"[economie, emploi, carriere, vie-professionnel...",None,None,None,None,None,NaN,2022-11-30 23:25:43,tv


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 243015 entries, 32967 to 107595
Data columns (total 26 columns):
 #   Column                 Non-Null Count   Dtype              
---  ------                 --------------   -----              
 0   url                    243015 non-null  object             
 1   news                   148607 non-null  object             
 2   news_publication       148607 non-null  object             
 3   publication_name       243015 non-null  object             
 4   publication_language   243015 non-null  object             
 5   news_publication_date  243015 non-null  datetime64[ns]     
 6   news_title             243015 non-null  object             
 7   news_keywords          120285 non-null  object             
 8   image                  146717 non-null  object             
 9   image_loc              239839 non-null  object             
 10  image_caption          158030 non-null  object             
 11  sitemap                243015 non-n

2 medias types: `tv` and `webpress`.

In [7]:
tv_medias = df[df["media_type"] == "tv"]["media"].unique()
tv_medias

array(['francetvinfo', 'bfmtv'], dtype=object)

In [8]:
webpress_medias = df[df["media_type"] == "webpress"]["media"].unique()
webpress_medias

array(['lefigaro', '20_minutes', 'investir.lesechos', 'liberation',
       'lesechos', 'lamarseillaise', 'lexpress', 'letelegramme',
       'le_point', 'lequipe', 'nouvel_obs', 'lemonde', 'lopinion'],
      dtype=object)

## 1. Label sitemap data

Le but est extraire en premier lieu les sections que nous utiliserons plus tard pour la définition des articles liés au climat. 

In [9]:
df["section"].head()

32967                                       [faits-divers]
32968    [economie, emploi, carriere, vie-professionnel...
32969          [politique, gouvernement-d-elisabeth-borne]
32970               [france, hauts-de-france, nord, lille]
32971    [economie, emploi, carriere, vie-professionnel...
Name: section, dtype: object

**Sans preprocess de la colonne**

In [35]:
# Get unique values
section_names = sorted(set(itertools.chain.from_iterable(df["section"])))
print(f"There are {len(section_names)} identified sections.")

There are 2948 identified sections.


In [36]:
# Save in file
with open("../../data_public/sitemap_dumps/section_names_without_preprocess.txt", "w") as fp:
    fp.write("\n".join(section_names))


In [13]:
# Identify climate-related sections
climate_sections = [
    "climat",
    "cop",
    "crise-climatique",
    "ecologie",
    "eelv",
    "ecosysteme",
    "empreinte-carbone",
    "energie",
    "energie-environnement",
    "energie-petrole-nucleaire-renouvelables-geopolitique",
    "energies",
    "environnement",
    "environnement-et-sante",
    "europe-ecologie-les-verts",
    "financement-de-travaux-et-renovation-energetique",
    "leolien-au-sein-de-loudeac-communaute",
    "leolien-autour-de-guingamp",
    "nucleaire",
    "nucleaire-iranien",
    "parc-eolien-en-baie-de-saint-brieuc-des-annees-de-tensions",
    "plan-de-sobriete",
    "planete",
    "planete-locale",
    "pollution-air",
    "renovation-travaux",
    "sur-le-green",
]


**Avec preprocess**

Les sections sont sous forme de liste, et pour la plupart, on trouve plusieurs niveaux : `[economie, social]`, `[weekend, business-story]`, `[sports, football, equipe-de-france]`,`[sport, article]`, `[environnement, climat]`, `[industrie-services, tourisme-transport]`, `[finistere, plouigneau]`, `[sport, football]`. Les listes semblnt hierarchisées, c'est-à-dire que le premier élement de la liste regroupe le deuxième élément etc.

Nous pouvons pour réduire le nombre de sections en se contentant du premier niveau : 
- `[sports, football, equipe-de-france]` --> `sports`, 
- `[sport, football] ` --> `sport`

In [37]:
def preprocess_section(df: pd.DataFrame) -> pd.Series:
    """Get first element of the list as "section" column.
    For example `[sports, football, equipe-de-france]` --> `sports`

    Args:
        df (pd.DataFrame): data with a "section" column in a list format

    Returns:
        pd.Series: each element is the first element of each list in the "section" column
    """
    df_section = df["section"].apply(lambda x: x[0])
    return df_section

In [38]:
df_section = preprocess_section(df) 

In [39]:
# Get unique values
section_names_with_preprocess = sorted(df_section.unique())
print(f"There are {len(section_names_with_preprocess)} identified sections.")

There are 391 identified sections.


In [41]:
# Save in file
with open("../../data_public/sitemap_dumps/section_names_with_preprocess.txt", "w") as fp:
    fp.write("\n".join(section_names_with_preprocess))


A la main, on identifie les sections qui pourraient être en rapport avec des sujets climat.

Attention : subjectif, et parfois ne parle pas de climat, donc à vérifier à la main aussi.

In [ ]:
climate_sections = [
    "climat",
    "ecologie",
    "energies",
    "environnement",
    "planete"
]

On va vérifier à la main si les sections identifiées comme climat sont bien associées à des articles mentionnant un sujet climat. Pour cela, on filtre les articles dont les sections appartiennent aux sections climat identifiées. On enregistre dans `climate_news_title.csv` les titres d'articles et leurs sections : cela nous servira comme base pour la vérification des vrais et faux positifs, et modifier si besoin la liste `climate_sections`.

In [67]:
def check_list_item_is_in_list(sections, climate_sections=climate_sections):
    return any(x in climate_sections for x in sections)

climate_mask = df["section"].apply(check_list_item_is_in_list)
df_climate = df[climate_mask][["news_title", "section"]]

In [68]:
df_climate.head()

,news_title,section
32975,Crise énergétique : les bouchers touchés de pl...,"[economie, energie]"
32979,"Environnement : pour le compost, des sacs pas ...","[monde, environnement]"
32989,Crise énergétique : une commune des Alpes-Mari...,"[economie, energie]"
32991,Rénovation énergétique : le gouvernement souha...,"[economie, energie]"
32997,"""Jusqu'à 10°C dans les classes"" : face à la fl...","[economie, energie]"


In [69]:
df_climate.shape

(2233, 2)

In [70]:
df_climate.to_csv("../../data_public/sitemap_dumps/climate_news_title.csv")

Une fois la liste des sections liées au sujet climat fixée, on associe chaque section à un label :
- 1 si liée au climat
- 0 sinon

In [71]:
# Map section to 1 if climate-related, else 0
section_label_mapping = {
    section: (1 if section in climate_sections else 0) for section in section_names
}


In [73]:
# Save to json
with open("../../data_public/sitemap_dumps/section_label_mapping.json", "w") as fp:
    json.dump(section_label_mapping, fp, indent=4)